In [1]:
import os
import pandas as pd
import numpy as np
from scipy.optimize import linear_sum_assignment
import datetime as dt
from collections import defaultdict, Counter
from tqdm import tqdm
import matplotlib.pyplot as plt
import datetime as dt
plt.rcParams['figure.figsize'] = [16, 10]
plt.rcParams['font.size'] = 16
import seaborn as sns

In [2]:
submission = pd.read_csv('/Users/suzukishinji/kaggle/christmas/sample_submission_random_v2.csv')

In [15]:
CHILD_PREF = pd.read_csv('/Users/suzukishinji/kaggle/christmas/child_wishlist_v2.csv', header=None).drop(0, 1).values
GIFT_PREF = pd.read_csv('/Users/suzukishinji/kaggle/christmas/gift_goodkids_v2.csv', header=None).drop(0, 1).values

In [26]:
N_CHILDREN = 1000000
N_GIFT_TYPE = 1000
N_GIFT_QUANTITY = 1000
N_GIFT_PREF = 100
N_CHILD_PREF = 1000
TWINS = int(0.04 * N_CHILDREN)
Triplets = int(0.005 * N_CHILDREN)

In [27]:
GIFT_HAPPINESS = (1. / (2 * N_GIFT_PREF)) * np.ones(shape=(N_GIFT_TYPE,N_CHILDREN),dtype=np.float32)
for g in range(N_GIFT_TYPE):
    for i, c in enumerate(GIFT_PREF[g]):
        GIFT_HAPPINESS[g,c] = -1. * (N_GIFT_PREF - i) / N_GIFT_PREF

CHILD_HAPPINESS = (1. / (2 * N_CHILD_PREF)) * np.ones(shape=(N_CHILDREN,N_GIFT_TYPE),dtype=np.float32)
for c in range(N_CHILDREN):
    for i, g in enumerate(CHILD_PREF[c]):
        CHILD_HAPPINESS[c,g] = -1. * (N_CHILD_PREF - i) / N_CHILD_PREF

GIFT_IDS = np.array([[g] * N_GIFT_QUANTITY for g in range(N_GIFT_TYPE)]).flatten()

In [28]:
def my_avg_normalized_happiness(pred):
    total_child_happiness = 0
    total_gift_happiness = np.zeros(1000)
    for c, g in pred:
        total_child_happiness +=  -CHILD_HAPPINESS[c,g]
        total_gift_happiness[g] += -GIFT_HAPPINESS[g,c]
    nch = total_child_happiness / N_CHILDREN
    ngh = np.mean(total_gift_happiness) / 1000
    print('normalized child happiness', nch)
    print('normalized gift happiness', ngh)
    return nch + ngh

In [29]:
def optimize_block(child_block, current_gift_ids):
    gift_block = current_gift_ids[child_block]
    C = np.zeros((BLOCK_SIZE, BLOCK_SIZE))
    for i in range(BLOCK_SIZE):
        c = child_block[i]
        for j in range(BLOCK_SIZE):
            g = GIFT_IDS[gift_block[j]]
            C[i, j] = CHILD_HAPPINESS[c,g] + GIFT_HAPPINESS[g,c]
    row_ind, col_ind = linear_sum_assignment(C)
    return (child_block[row_ind], gift_block[col_ind])

In [30]:
BLOCK_SIZE = 400
INITIAL_SUBMISSION = '/Users/suzukishinji/kaggle/christmas/sample_submission_random_v2.csv'
N_BLOCKS = (N_CHILDREN - TWINS - Triplets) / BLOCK_SIZE
print('Block size: {}, n_blocks {}'.format(BLOCK_SIZE, N_BLOCKS))

Block size: 4000, n_blocks 238.75


In [31]:
subm = pd.read_csv(INITIAL_SUBMISSION)
initial_anh = my_avg_normalized_happiness(subm[['ChildId', 'GiftId']].values.tolist())
print(initial_anh)
subm['gift_rank'] = subm.groupby('GiftId').rank() - 1
subm['gift_id'] = subm['GiftId'] * 1000 + subm['gift_rank']
subm['gift_id'] = subm['gift_id'].astype(np.int64)
current_gift_ids = subm['gift_id'].values

normalized child happiness 0.0951774029867
normalized gift happiness -0.0087123548812
0.0864650481055


In [23]:
start_time = dt.datetime.now()
for i in range(1):
    child_blocks = np.split(np.random.permutation(range(TWINS, N_CHILDREN)), N_BLOCKS)
    for child_block in tqdm(child_blocks[:500]):
        cids, gids = optimize_block(child_block, current_gift_ids=current_gift_ids)
        current_gift_ids[cids] = gids
    subm['GiftId'] = GIFT_IDS[current_gift_ids]
    anh = my_avg_normalized_happiness(subm[['ChildId', 'GiftId']].values.tolist())
    end_time = dt.datetime.now()
    print(i, anh, (end_time-start_time).total_seconds())
subm[['ChildId', 'GiftId']].to_csv("submit_1226_1.csv",index = False)

  0%|          | 0/240 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [24]:
subm[['ChildId', 'GiftId']].to_csv("submit_1226_1.csv",index = False)

In [25]:
submit_1 = pd.read_csv('/Users/suzukishinji/kaggle/christmas/submit_1226_1.csv')
submit_1.head(10)

,ChildId,GiftId
0,0,669
1,1,669
2,2,669
3,3,8
4,4,8
5,5,8
6,6,689
7,7,689
8,8,689
9,9,891
